In [1]:
from rdflib import Graph, URIRef, BNode, Literal, Namespace
from rdflib.namespace import RDF, FOAF, RDFS, SKOS, DCTERMS
import pandas as pd
import re
import string

In [4]:
df = pd.read_csv('Accidental_Drug_Related_Deaths_2012-2018.csv')
ds = Namespace("https://data.ct.gov/resource/deaths/")
socrata = Namespace("http://www.socrata.com/rdf/terms#")
dcat = Namespace("http://www.w3.org/ns/dcat#")
ods = Namespace("http://open-data-standards.github.com/2012/01/open-data-standards#")
geo =  Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
skos = Namespace("http://www.w3.org/2004/02/skos/core#")
dsbase = Namespace("https://data.ct.gov/resource/")
usps = Namespace("http://www.w3.org/2000/10/swap/pim/usps#")


g = Graph()
g.bind('ds', ds)
g.bind('usps', usps)
g.bind('dsbase', dsbase)
g.bind('geo', geo)
for index, row in df.iterrows():
    g.add( (URIRef(row[0]), RDF.type , dsbase.Person ))
    for col in df.columns:
        if (col == "Date"):
            row[col] = str(row[col])[0:10]
        if pd.isna(row[col]) == False:
            if (col == "InjuryCityGeo" or col == "DeathCityGeo" or col == "ResidenceCityGeo"):
                coords = re.search('\(([^)]+)', row[col]).group(1)
                coords = coords.split(",")
                city_state = ((row[col].split("(")[0]).rstrip()).split(",")
                g.add( (URIRef(row[0]), URIRef(ds+col), URIRef(geo+row[0]+col) ))
                g.add((URIRef(geo+row[0]+col), RDF.type, URIRef(geo.SpatialThings)))
                if len(city_state) == 1:
                    g.add((URIRef(geo+row[0]+col), usps.state, Literal(city_state[0])))
                if len(city_state) == 2:
                    g.add((URIRef(geo+row[0]+col), usps.city, Literal(city_state[0])))
                    g.add((URIRef(geo+row[0]+col), usps.state, Literal(city_state[1])))
                g.add((URIRef(geo+row[0]+col), geo.latitude, Literal(coords[0])))
                g.add((URIRef(geo+row[0]+col), geo.longitude, Literal(coords[1])))

            else:
                g.add( (URIRef(row[0]), URIRef(ds+col), Literal(row[col]) ))

g.serialize("test.rdf", format="pretty-xml")